In [7]:
!pip install vedo trimesh open3d --quiet

In [19]:
import numpy as np
import vedo
import trimesh
import open3d as o3d
import random
import os
import csv

class GeneradorEscenas3D:
    """Clase para la generación y visualización de escenas 3D"""

    def __init__(self):
        """Constructor de la clase"""
        # Crear carpeta para guardar los archivos
        self.directorio_salida = "output_3d"
        os.makedirs(self.directorio_salida, exist_ok=True)

        # Inicializar colecciones
        self.puntos = None
        self.objetos = []
        self.visualizador = None

    def generar_coordenadas(self, cantidad=10, limite=5):
        """
        Genera coordenadas aleatorias para los puntos en el espacio 3D

        Args:
            cantidad: Número de puntos a generar
            limite: Rango de coordenadas (-limite, limite)

        Returns:
            Array numpy con las coordenadas generadas
        """
        print(f"Generando {cantidad} puntos en el espacio...")

        coordenadas = []
        for _ in range(cantidad):
            x = random.uniform(-limite, limite)
            y = random.uniform(-limite, limite)
            z = random.uniform(-limite, limite)
            coordenadas.append([x, y, z])

        self.puntos = np.array(coordenadas)
        print(f"Puntos generados: {len(self.puntos)}")
        return self.puntos

    def crear_objetos(self):
        """
        Crea objetos 3D (primitivas) basados en los puntos generados

        Returns:
            Lista de objetos vedo generados
        """
        if self.puntos is None:
            print("Error: Primero debe generar coordenadas")
            return []

        print("Creando objetos 3D...")
        objetos = []
        colores = vedo.colors.cmaps_names

        for i, punto in enumerate(self.puntos):
            tipo_objeto = i % 3  # Determina el tipo de objeto
            color = random.choice(colores)

            if tipo_objeto == 0:  # Esferas
                radio = 0.2 + (i * 0.05)  # Radio variable
                esfera = vedo.Sphere(pos=punto, r=radio, c=color)
                objetos.append(esfera)

            elif tipo_objeto == 1:  # Cubos
                lado = 0.3 + (i * 0.04)  # Tamaño variable
                cubo = vedo.Cube(pos=punto, side=lado, c=color)
                objetos.append(cubo)

            else:  # Cilindros
                radio = 0.15 + (i * 0.03)
                altura = 0.5 + (i * 0.1)
                # Orientación aleatoria para el cilindro
                punto_final = punto + np.array([
                    random.uniform(-1, 1),
                    random.uniform(-1, 1),
                    altura
                ])
                cilindro = vedo.Cylinder(pos=[punto, punto_final], r=radio, c=color)
                objetos.append(cilindro)

        self.objetos = objetos
        print(f"Objetos creados: {len(self.objetos)}")
        return self.objetos

    def visualizar(self):
        """Muestra la escena 3D en una ventana"""
        if not self.objetos:
            print("No hay objetos para visualizar")
            return

        print("Mostrando escena 3D...")
        escena = vedo.Plotter()
        for obj in self.objetos:
            escena.add(obj)

        # Mostrar ejes de coordenadas
        escena.show(axes=1)

    def exportar(self, formato="obj"):
        """
        Exporta la escena en el formato especificado

        Args:
            formato: Formato de exportación (obj, ply, etc.)

        Returns:
            Ruta del archivo guardado
        """
        if not self.objetos:
            print("No hay objetos para exportar")
            return None

        # Contar archivos existentes para evitar sobrescribir
        contador = len([
            f for f in os.listdir(self.directorio_salida)
            if f.startswith("escena_") and f.endswith(f".{formato}")
        ])

        # Crear una malla combinada
        malla_combinada = vedo.merge(self.objetos)

        # Guardar archivo
        archivo_salida = f"{self.directorio_salida}/escena_{contador}.{formato}"
        vedo.write(malla_combinada, archivo_salida)

        print(f"Escena exportada como {archivo_salida}")
        return archivo_salida

    def generar_csv_ejemplo(self):
        """
        Genera un archivo CSV de ejemplo con coordenadas y parámetros

        Returns:
            Array numpy con las coordenadas generadas desde el CSV
        """
        print("Generando archivo CSV de ejemplo...")

        # Crear datos aleatorios
        n_filas = 15
        datos = np.random.rand(n_filas, 6)  # x,y,z,tipo,tamaño,color_idx

        # Guardar CSV
        archivo_csv = f"{self.directorio_salida}/datos_ejemplo.csv"
        encabezados = ["x", "y", "z", "tipo", "tamaño", "color_idx"]

        with open(archivo_csv, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(encabezados)
            writer.writerows(datos)

        print(f"CSV generado: {archivo_csv}")
        return self.cargar_desde_csv(archivo_csv)

    def cargar_desde_csv(self, archivo_csv):
        """
        Carga coordenadas desde un archivo CSV

        Args:
            archivo_csv: Ruta al archivo CSV

        Returns:
            Array numpy con las coordenadas cargadas
        """
        print(f"Cargando datos desde {archivo_csv}...")

        # Leer datos del CSV
        datos = np.loadtxt(archivo_csv, delimiter=",", skiprows=1)

        # Crear objetos desde los datos
        self.objetos = []
        colores = vedo.colors.cmaps_names

        for fila in datos:
            x, y, z, tipo, tamaño, color_idx = fila
            # Escalar coordenadas a un rango adecuado
            punto = [x*10-5, y*10-5, z*10-5]

            # Determinar color
            idx_color = int(color_idx * len(colores)) % len(colores)
            color = colores[idx_color]

            # Crear objeto según el tipo
            if tipo < 0.33:  # Esfera
                obj = vedo.Sphere(pos=punto, r=tamaño, c=color)
            elif tipo < 0.66:  # Cubo
                obj = vedo.Cube(pos=punto, side=tamaño, c=color)
            else:  # Cilindro
                punto_final = punto + np.array([0, 0, tamaño*2])
                obj = vedo.Cylinder(pos=[punto, punto_final], r=tamaño/2, c=color)

            self.objetos.append(obj)

        print(f"Objetos creados desde CSV: {len(self.objetos)}")
        return np.array([fila[:3] for fila in datos])  # Devolver solo coordenadas (x,y,z)

def main():
    """Función principal que ejecuta la demostración del programa"""
    print("\n=== GENERADOR DE ESCENAS 3D ===\n")

    # Crear instancia del generador
    generador = GeneradorEscenas3D()

    # Generar escena con puntos aleatorios
    print("\n--- ESCENA 1: PUNTOS ALEATORIOS ---")
    generador.generar_coordenadas(15,5)
    generador.crear_objetos()
    generador.visualizar()
    generador.exportar(formato="obj")

    # Generar escena desde CSV
    print("\n--- ESCENA 2: DATOS DESDE CSV ---")
    generador.generar_csv_ejemplo()
    generador.visualizar()
    generador.exportar(formato="obj")

    print("\n¡Proceso completado! Los archivos están disponibles en la carpeta:", generador.directorio_salida)

# Si el script se ejecuta directamente
if __name__ == "__main__":
    main()


=== GENERADOR DE ESCENAS 3D ===


--- ESCENA 1: PUNTOS ALEATORIOS ---
Generando 15 puntos en el espacio...
Puntos generados: 15
Creando objetos 3D...
Objetos creados: 15
Mostrando escena 3D...
Escena exportada como output_3d/escena_0.obj

--- ESCENA 2: DATOS DESDE CSV ---
Generando archivo CSV de ejemplo...
CSV generado: output_3d/datos_ejemplo.csv
Cargando datos desde output_3d/datos_ejemplo.csv...
Objetos creados desde CSV: 15
Mostrando escena 3D...
Escena exportada como output_3d/escena_1.obj

¡Proceso completado! Los archivos están disponibles en la carpeta: output_3d
